In [1]:
import pdfplumber
from pathlib import Path

def extract_pdf_text(pdf_path: str) -> str:
    """从 PDF 文件中提取全部文本（不落地保存 .txt），直接返回字符串。"""
    pdf_path = Path(pdf_path)
    assert pdf_path.exists(), f"文件不存在: {pdf_path}"
    
    all_text = []
    with pdfplumber.open(pdf_path) as pdf:
        for i, page in enumerate(pdf.pages, start=1):
            text = page.extract_text() or ""
            all_text.append(text)
    
    return "\n".join(all_text)

print("✅ extract_pdf_text 已定义，可以直接用于提取 PDF 文本。")


✅ extract_pdf_text 已定义，可以直接用于提取 PDF 文本。


In [2]:
from KG_tools.extract_kg_stream import extract_kg_stream
from KG_tools.extract_pdf_text import extract_pdf_text
import json

pdf_path = "D:\\Desktop\\现代软件工程\\作业\\实践营\\KG\\KG_files\\专利法2021.pdf"  # 例如："./laws/sample.pdf"

# 1. 从 PDF 中提取全部文本（只在内存中操作，不生成 txt 文件）
pdf_text = extract_pdf_text(pdf_path)
print("📄 从 PDF 中提取的文本长度:", len(pdf_text))
print("----- 以下为部分提取内容预览 -----")
print(pdf_text[:1179])  # 预览前 1179 个字符

✅ extract_pdf_text 已定义，可以直接用于提取 PDF 文本。
📄 从 PDF 中提取的文本长度: 1179
----- 以下为部分提取内容预览 -----
《中华人民共和国专利法（2021年修订版全文）》
（2021年6月1日起施行）
第一章 总则
第一条 为了保护发明创造的专利权，鼓励发明创造的公开和应用，促进科学技术进步和创新，推动
经济社会发展，制定本法。
第二条 本法所称发明创造是指发明、实用新型和外观设计。
第三条 国务院专利行政部门负责全国的专利工作；县级以上地方人民政府管理专利工作的部门负责
本行政区域内的专利工作。
第四条 发明和实用新型专利申请由国务院专利行政部门负责受理、审查和批准。
第五条 探索违反法律、社会公德或妨害公共利益的发明创造，不授予专利权。
第六条 执行本单位任务或者主要利用本单位物质技术条件所完成的发明创造为职务发明创造。
第七条 发明创造申请专利的权利属于发明人或者设计人，单位可以与个人约定权利归属。
第八条 两个以上单位或者个人共同完成的发明创造，专利申请权和专利权由约定决定。
第二章 专利的授予条件
第二十二条 发明和实用新型应当具备新颖性、创造性和实用性。
第二十三条 外观设计应当与已有设计明显不同，不得侵害他人在先权利。
第二十四条 某些公开情形不丧失新颖性。
第二十五条 科学发现、智力活动方法、疾病诊断方法等不得授予专利权。
第三章 专利申请
第二十六条 申请发明或实用新型应提交请求书、说明书及必要图纸。
第二十七条 申请外观设计应提交设计图片或照片及简要说明。
第三十条 申请日以国务院专利行政部门收到申请文件之日为准。
第四章 专利审查
第三十四条 发明专利申请进行初步审查。
第三十五条 自申请日起三年内可提出实质审查请求。
第三十九条 审查合格的，授予发明专利权。
第四十条 实用新型和外观设计经初步审查合格即可授予专利权。
第五章 专利权的期限与保护
第四十二条 发明专利期限为20年，实用新型和外观设计为15年，自申请日起计算。
第四十三条 专利权人应缴纳年费。
第四十四条 出现放弃、期满、未按期缴费等情形，专利权终止。
第四十八条 未经许可实施专利的，属于侵权。
第五十条 专利权人或利害关系人可请求行政保护。
第五十二条 专利侵权纠纷由人民法院处理。
第五十五条 侵权人应赔偿损失，可适用惩罚性赔

In [3]:
kg = extract_kg_stream(pdf_text) # 抽取知识图谱
print("\n=== 最终解析结果（美化输出） ===")
print(json.dumps(kg, ensure_ascii=False, indent=2))

Response:
We are given a chunk of the Patent Law of China (2021 revised version). We need to extract entities and relations strictly following the rules.

 Steps:
 1. Identify structural entities: Chapters and Articles.
    - Chapters: "第一章", "第二章", ... "第十章". We see from text: 第一章, 第二章, 第三章, 第四章, 第五章, 第六章, 第七章, 第八章, 第九章, 第十章.
    - Articles: "第一条", "第二条", ... up to "第七十六条". The text lists articles from 第一条 to 第七十六条, but note some are skipped (e.g., 第二十八条, 第二十九条 are missing).

 2. For each article, extract semantic entities (Concepts, Rights, Definitions, Entity) from the body of the article.

 3. Rules to avoid structural entities as semantic ones: Only the body text (after the article number) is used for semantic extraction.

 4. We must create:
    - Law entity for the entire law (since it's the first line).
    - Chapter entities for each chapter (with names from the text: "总则", "专利的授予条件", etc.)
    - Article entities for each article.

 5. Relations:
    - Law HAS_CHAPTER Chapter


In [4]:
# ========= 规则增强逻辑：自动 split/attach body/enrich KG ==========

import re

# 1️⃣ 按“第X条”切分 PDF 文本
def split_articles(text: str):
    pattern = r'(第[一二三四五六七八九十百零〇0-9]+条)'
    parts = re.split(pattern, text)
    article_map = {}

    for i in range(1, len(parts), 2):
        label = parts[i].strip()
        body = parts[i+1] if i+1 < len(parts) else ""
        body = re.sub(pattern, "", body).strip()
        article_map[label] = body

    return article_map

article_text_map = split_articles(pdf_text)


# 2️⃣ 给 KG 中所有 Article 节点挂上正文（extra.body）
for ent in kg["content"]["entities"]:
    if ent.get("node_type") == "Article":
        label = ent.get("name")
        if label in article_text_map:
            ent.setdefault("extra", {})["body"] = article_text_map[label]


# 3️⃣ 针对“应提交 XXX”模式的条文，自动补实体+关系
def enrich_kg_with_submission_rules(kg):
    entities = kg["content"]["entities"]
    relations = kg["content"]["relations"]
    uid2ent = {e["uid"]: e for e in entities}

    def ensure_concept(name, extra=None):
        uid = f"Concept:{name}"
        if uid not in uid2ent:
            ent = {
                "uid": uid,
                "node_type": "Concept",
                "name": name,
                "extra": extra or {}
            }
            uid2ent[uid] = ent
            entities.append(ent)
        return uid

    for ent in entities:
        if ent.get("node_type") == "Article":
            article_uid = ent["uid"]
            body = ent.get("extra", {}).get("body", "")
            if not body:
                continue

            # 只处理含“应提交”的条文
            if "应提交" not in body:
                continue

            # 抽“申请 XX 应提交”的 XX —— APPLIES_TO_OBJECT
            m_obj = re.search(r"申请(.+?)应提交", body)
            if m_obj:
                obj_name = m_obj.group(1).strip(" ，。、“”")
                obj_uid = ensure_concept(obj_name)
                relations.append({
                    "from_uid": article_uid,
                    "to_uid": obj_uid,
                    "rel_type": "APPLIES_TO_OBJECT"
                })

            # 抽“应提交 A、B、C” —— REQUIRES_DOC
            m_docs = re.search(r"应提交(.+?)[。;；]", body)
            if m_docs:
                docs_str = m_docs.group(1)
                doc_names = re.split(r"[、和及，,]", docs_str)
                for name in doc_names:
                    name = name.strip(" ，。、“”")
                    if not name:
                        continue
                    doc_uid = ensure_concept(name, extra={"category": "application_document"})
                    relations.append({
                        "from_uid": article_uid,
                        "to_uid": doc_uid,
                        "rel_type": "REQUIRES_DOC"
                    })
    return kg


# 执行规则增强
kg = enrich_kg_with_submission_rules(kg)
print("🔧 已自动增强 KG：挂正文 + 自动抽实体 + 自动建关系。")


🔧 已自动增强 KG：挂正文 + 自动抽实体 + 自动建关系。


In [5]:
# 选择输出文件夹并保存为 JSON 文件 （其会依据解析的KG自己命名）
import os
from KG_tools.save_kg_json import save_kg_json

OUTPUT_DIR = "D:\\Desktop\\现代软件工程\\作业\\实践营\\KG\\KG_files\\KG_json_test"
os.makedirs(OUTPUT_DIR, exist_ok=True)

article_uid = None
for ent in kg["content"]["entities"]:
    if ent["node_type"] == "Article":
        article_uid = ent["uid"]  # 比如 "Article:第10条"
        break

if article_uid is None:
    filename = "unknown_article.json"
else:
    # 简单处理下文件名里的冒号
    safe_uid = article_uid.replace(":", "_")
    filename = f"{safe_uid}.json"   # Article_第10条.json

save_kg_json(kg, OUTPUT_DIR, filename)

已保存到: D:\Desktop\现代软件工程\作业\实践营\KG\KG_files\KG_json_test\Article_第一条.json


In [13]:
from neo4j import GraphDatabase

NEO4J_URI = "neo4j://127.0.0.1:7687"
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = "20011127"  # 换成你 Neo4j Desktop 里设置的

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))
print("Neo4j driver initialized.")

Neo4j driver initialized.


In [14]:
from KG_tools.upsert_entities import upsert_entities
from KG_tools.upsert_relations import upsert_relations

entities = kg["content"]["entities"]
relations = kg["content"]["relations"]

with driver.session() as session:
    session.execute_write(upsert_entities, entities)
    session.execute_write(upsert_relations, relations)

print("当前这条法条已写入 Neo4j。")

当前这条法条已写入 Neo4j。


In [15]:
from KG_tools.import_json_to_neo4j import import_json_to_neo4j

JSON_DIR = "D:\\Desktop\\现代软件工程\\作业\\实践营\\KG\\KG_files\\KG_json_test"  # ← 你的目录
import_json_to_neo4j(JSON_DIR)

检测到 JSON 文件： ['Article_第10条.json', 'Article_第一条.json']

>>> 处理文件：D:\Desktop\现代软件工程\作业\实践营\KG\KG_files\KG_json_test\Article_第10条.json

>>> 处理文件：D:\Desktop\现代软件工程\作业\实践营\KG\KG_files\KG_json_test\Article_第一条.json

🎉 所有 JSON 文件已成功写入 Neo4j！
